In [1]:
import pandas as pd
import numpy as np
import datetime

import warnings
warnings.filterwarnings("ignore")
import glob
import ee
import pickle
ee.Initialize()


ModuleNotFoundError: No module named 'ee'

In [2]:

def maskS2clouds(image):

  qa = image.select('QA60');
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));

  return image.updateMask(mask);
  

def NDWI_function(image):
  NDWI = image.expression(
      '(B03-B08)/(B03+B08)', {
        'B03': image.select('B3'),
        'B08': image.select('B8')

  }).rename('NDWI');
  return image.addBands(NDWI);

    
def TBVI1_function(image):

  TBVI1 = image.expression(
    'B05/(B05+B02)', {
      'B02': image.select('B2'),
      'B05': image.select('B5')

  }).rename('TBVI1');
  return image.addBands(TBVI1);


    
def NDVI_G_function(image):

  NDVI_G = image.expression(
  '(B8A-B03)/(B8A+B03)', {
  'B03': image.select('B3'),
  'B8A': image.select('B8A')

  }).rename('NDVI_G');
  return image.addBands(NDVI_G)

    
def SR_N_function(image):
  SR_N = image.expression(
  'B8A/B02', {
  'B02': image.select('B2'),
  'B8A': image.select('B8A')

  }).rename('SR_N');
  return image.addBands(SR_N);

def SR_n2_function(image):
  SR_n2 = image.expression(
      'B08/B03', {
        'B03': image.select('B3'),
        'B08': image.select('B8')
  
  }).rename('SR_n2');
  return image.addBands(SR_n2)

def NDVI_function(image):

  NDVI = image.expression(
      '(B08-B04)/(B08+B04)', {
        'B04': image.select('B4'),
        'B08': image.select('B8')

  }).rename('NDVI');
  return image.addBands(NDVI);

def maskS2clouds_min_max(image):
  
  qa = image.select('QA60');

  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;

  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));

  img = image.updateMask(mask).divide(10000);
  img_time = img.set("system:time_start", image.get("system:time_start"))
  return img_time

def addNDVI(image):
  ndvi_s = image.normalizedDifference(['B8', 'B4']).rename('NDVI');
  ndvi = ndvi_s.set("system:time_start", image.get("system:time_start"));
  return image.addBands(ndvi)

# def clipToCol(image):
#   return image.clipToCollection(point)


# def masker(image):
#   mask1 = greenest.select('NDVI').gt(0.3);
#   mask2 = min.select('NDVI').lt(0.6);
#   mask3=image.select('NDVI').gt(0.05)
#   mask4=image.select('NDVI').lt(0.3)
#   return image.updateMask(mask1).updateMask(mask2).updateMask(mask3).updateMask(mask4)



In [6]:
df = pd.read_csv("../dump/lat_lons.csv").reset_index()

points_lmt = 1000

df.loc[:points_lmt, 'Date'] = pd.to_datetime(df.loc[:points_lmt,'Date'].apply(lambda x: x.split("T")[0]), format = '%Y-%m-%d')

df_tmp = df.loc[:points_lmt, :]

In [7]:

def masker(greenest, min):
    def wrap(image):                
        mask1 = greenest.select('NDVI').gt(0.3);
        mask2 = min.select('NDVI').lt(0.6);
        mask3=image.select('NDVI').gt(0.05)
        mask4=image.select('NDVI').lt(0.3)
        return image.updateMask(mask1).updateMask(mask2).updateMask(mask3).updateMask(mask4)
    return wrap

def clipToCol(point):
    def wrap(image):
            
        return image.clipToCollection(point)
    return wrap

def predictor_time(row):
    
    current_date = row['Date']

    start_date = ee.Date(current_date - datetime.timedelta(days = 16))

    end_date= ee.Date(current_date)
    point = ee.FeatureCollection(ee.Geometry.Point([row['Longitude'], row['Latitude']]));

    s2_sr = ee.ImageCollection("COPERNICUS/S2_SR")

    s2_sr_filter = s2_sr.filterBounds(point).filterDate(start_date,end_date).map(maskS2clouds);
                    
    sentinel2 = ee.ImageCollection(s2_sr_filter.mosaic().set("system:time_start", ee.Image(s2_sr_filter.first()).get("system:time_start")))

    indices_collection = sentinel2.map(NDWI_function).map(TBVI1_function).map(NDVI_G_function).map(SR_N_function).map(SR_n2_function).map(NDVI_function);
    

    s2_NDVI = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(point).filterDate('2020-01-01', '2020-12-31')\
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 50)).map(maskS2clouds_min_max).map(addNDVI).select("NDVI").map(clipToCol(point));

    greenest = s2_NDVI.qualityMosaic('NDVI')
    min = s2_NDVI.min()

    predictor_bands = indices_collection.map(masker(greenest, min))

    df = pd.DataFrame(predictor_bands.select('B8', 'B4', 'B5', 'B11','B9', 'B1', 'SR_n2', 'SR_N', 'TBVI1', 'NDWI', 'NDVI_G')\
        .getRegion(point, 10).getInfo())
    
    df, df.columns = df[1:] , df.iloc[0] 
    df = df.drop(["id"], axis =1)

    return df.values[0]


# asap = df_tmp.apply(predictor_time, axis = 1, result_type='expand', )

In [8]:
df_tmp = df_tmp.dropna()

In [9]:
asap = df_tmp.apply(predictor_time, axis = 1,  result_type='expand')


In [10]:
asap

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,74.934721,33.800595,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,74.808508,33.800864,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,74.935530,33.800864,1613454580078,1376.0,1136.0,1374.0,2015.0,1740.0,996.0,1.176068,1.312825,0.543513,-0.080911,0.128492
3,74.935799,33.800415,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,74.935619,33.800864,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,74.935979,32.658926,1614318595322,2234.0,1504.0,1524.0,2333.0,2319.0,686.0,1.679699,2.168932,0.596711,-0.253648,0.253648
997,74.808508,33.800954,1614318580670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,74.935799,33.801134,1614578376668,1660.0,1306.0,1477.0,2358.0,1896.0,1010.0,1.381032,1.724138,0.606820,-0.160028,0.157083
999,74.935979,33.800685,1614578376668,2496.0,1482.0,1868.0,2666.0,1944.0,1032.0,1.827233,2.332061,0.640604,-0.292595,0.282940


In [11]:
asap['sys_index'] = df_tmp['index']

In [ ]:
asap

In [14]:
asap.columns = ['longitude', 'latitude', 'time' ,'B8', 'B4', 'B5', 'B11', 'B9', 'B1', 'SR_n2','SR_N', 'TBVI1', 'NDWI', 'NDVI_G', 'sys_index']

In [15]:
asap

,longitude,latitude,time,B8,B4,B5,B11,B9,B1,SR_n2,SR_N,TBVI1,NDWI,NDVI_G,sys_index
0,74.934721,33.800595,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,74.808508,33.800864,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,74.935530,33.800864,1613454580078,1376.0,1136.0,1374.0,2015.0,1740.0,996.0,1.176068,1.312825,0.543513,-0.080911,0.128492,2
3,74.935799,33.800415,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,74.935619,33.800864,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,74.935979,32.658926,1614318595322,2234.0,1504.0,1524.0,2333.0,2319.0,686.0,1.679699,2.168932,0.596711,-0.253648,0.253648,996
997,74.808508,33.800954,1614318580670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,997
998,74.935799,33.801134,1614578376668,1660.0,1306.0,1477.0,2358.0,1896.0,1010.0,1.381032,1.724138,0.606820,-0.160028,0.157083,998
999,74.935979,33.800685,1614578376668,2496.0,1482.0,1868.0,2666.0,1944.0,1032.0,1.827233,2.332061,0.640604,-0.292595,0.282940,999


In [19]:
asd

,B8,B4,B5,B11,B9,B1,SR_n2,SR_N,TBVI1,NDWI,NDVI_G
2,1376.0,1136.0,1374.0,2015.0,1740.0,996.0,1.176068,1.312825,0.543513,-0.080911,0.128492
5,2328.0,1390.0,1729.0,2462.0,1575.0,1013.0,1.796296,1.955106,0.603490,-0.284768,0.263008
6,1860.0,1406.0,1445.0,2208.0,1575.0,1013.0,1.319149,1.450549,0.531445,-0.137615,0.134438
7,1958.0,1628.0,1792.0,2415.0,1740.0,996.0,1.333787,1.547582,0.582954,-0.143024,0.149479
8,2120.0,1588.0,1731.0,2528.0,1740.0,996.0,1.545190,1.700000,0.594641,-0.214204,0.187685
...,...,...,...,...,...,...,...,...,...,...,...
994,2068.0,1544.0,1855.0,2923.0,1937.0,1030.0,1.540984,2.205911,0.646341,-0.212903,0.250489
995,1212.0,973.0,1080.0,2144.0,1944.0,1032.0,1.103825,1.344455,0.514531,-0.049351,0.110211
996,2234.0,1504.0,1524.0,2333.0,2319.0,686.0,1.679699,2.168932,0.596711,-0.253648,0.253648
998,1660.0,1306.0,1477.0,2358.0,1896.0,1010.0,1.381032,1.724138,0.606820,-0.160028,0.157083


In [23]:
soil_nuts = [ 'P', 'K', 'OC', 'N', 'pH']

input = asap[['B8', 'B4', 'B5', 'B11', 'B9', 'B1', 'SR_n2','SR_N', 'TBVI1', 'NDWI', 'NDVI_G']]

mylist = []
asd = input.dropna()
for i in soil_nuts:
    
    for nut, nut_slr in zip(glob.glob(f'../data/models/{i}.pkl'), glob.glob(f'../data/models/{i}*_slr.pkl')):
        with open(nut, 'rb') as file, open(nut_slr, 'rb') as file_slr:

            mymodel, model_scaler = pickle.load(file), pickle.load(file_slr)
            X_test = model_scaler.transform(asd)
            
            predictions = mymodel.predict(X_test).reshape(-1,)
            if i == 'OC':
                predictions = predictions.clip(0, 1)
            mylist.append(predictions)
mydf = pd.DataFrame(mylist).T
mydf.columns = [ 'P', 'K', 'OC', 'N', 'pH']
mydf.index = asd.index

# pd.concat([asd, mydf], axis = 1, ignore_index=True,)
pd.concat([asap, mydf], axis = 1, ignore_index=False) 


,longitude,latitude,time,B8,B4,B5,B11,B9,B1,SR_n2,SR_N,TBVI1,NDWI,NDVI_G,sys_index,P,K,OC,N,pH
0,74.934721,33.800595,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,74.808508,33.800864,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
2,74.935530,33.800864,1613454580078,1376.0,1136.0,1374.0,2015.0,1740.0,996.0,1.176068,1.312825,0.543513,-0.080911,0.128492,2,25.448950,269.034061,0.529348,273.946935,7.063188
3,74.935799,33.800415,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN
4,74.935619,33.800864,1613454580078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,74.935979,32.658926,1614318595322,2234.0,1504.0,1524.0,2333.0,2319.0,686.0,1.679699,2.168932,0.596711,-0.253648,0.253648,996,19.073650,205.816072,0.427051,234.071836,7.681413
997,74.808508,33.800954,1614318580670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,997,NaN,NaN,NaN,NaN,NaN
998,74.935799,33.801134,1614578376668,1660.0,1306.0,1477.0,2358.0,1896.0,1010.0,1.381032,1.724138,0.606820,-0.160028,0.157083,998,22.082608,251.391635,0.158886,213.991028,7.455972
999,74.935979,33.800685,1614578376668,2496.0,1482.0,1868.0,2666.0,1944.0,1032.0,1.827233,2.332061,0.640604,-0.292595,0.282940,999,17.568234,176.462880,0.556798,269.958701,7.981605


In [24]:
pd.concat([asap, mydf], axis = 1, ignore_index=False).to_csv("../dump/newAlgo_first1000.csv")

In [2]:
import geopandas as gpd

lon, lat = 75.338409 , 15.404190
wkts = [
f'POINT ({lon} {lat})'
]
s = gpd.GeoSeries.from_wkt(wkts)


In [27]:
geom = pd.read_csv("/data1/BKUP/micro_v2/s1_rvi/area/19525.csv", header=None, sep='\n')
gpd.GeoSeries.from_wkt(geom.iloc[:, 0])

0    POLYGON ((76.18816 13.71704, 76.18823 13.71489...
dtype: geometry

In [29]:
asd = s.buffer(0.0007).set_crs("EPSG:4326")#.to_csv("test_1.csv", index=False, header=False)

In [30]:
asd

0    POLYGON ((75.33911 15.40419, 75.33911 15.40412...
dtype: geometry